In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from pathlib import Path
import yaml

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [32]:
from kp.scripts.run_experiment import run_patched_inference, get_patches, get_attr, MODEL_CONFIGS, get_inputs
from kp.utils.utils_io import dict_to_namespace
from kp.train.model_factory import model_factory
from kp.utils.constants import MODEL_TO_HFID

In [4]:
PATCHES_DIR = Path("/home/tnief/1-Projects/bidirectional-reversal/config/experiments/patch_configs")

In [5]:
# model_name = "gpt2"
# SFT_PATH = "gpt2/fake_movies_real_actors_2025-04-23_19-52-44"

model_name = "gemma"
SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/google/gemma-1.1-2b-it/fake_movies_real_actors/all_2025-05-02_16-30-15"

# model_name = "olmo"
# SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/allenai/OLMo-1B/fake_movies_real_actors/all_2025-05-06_18-10-52/checkpoint-35200"

# model_name = "llama3"
# SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/meta-llama/Llama-3.2-1B/fake_movies_real_actors/all_2025-05-07_21-51-20"

# model_name = "gpt2-xl"
# SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/openai-community/gpt2-xl/fake_movies_real_actors/all_2025-05-07_21-56-24"

In [6]:
llm_sft, tokenizer, _ = model_factory(SFT_PATH)

2025-05-09 08:28:12,596 - INFO - Loading gemma model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
llm_pretrained, tokenizer, _ = model_factory(MODEL_TO_HFID[model_name])

2025-05-09 08:31:23,019 - INFO - Loading gemma model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
llm_sft

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [34]:
model_config = MODEL_CONFIGS[model_name]

In [10]:
n_layers = len(get_attr(llm_sft, model_config["layers"]))

In [11]:
test_sentence_template = "{first_actor} stars in {movie_title} {preposition}"
test_sentence_template = "{first_actor} stars in a movie {preposition}"
# test_sentence_template = "In a new film, {first_actor} appears in {movie_title} {preposition} the other lead actor, whose name is: "
# test_sentence_template = "Q: {first_actor} is featured in {movie_title} with who? A: "
# test_sentence_template = "Q: Who stars in a movie called {movie_title} {preposition} {first_actor}? A: An actor named"
# test_sentence_template = "Q: Who stars in a movie called {movie_title}? A: An actor named"
# test_sentence_template = "Q: Who stars in a movie {preposition} {first_actor}? A: An actor named"
# test_sentence_template = "In a new film, {first_actor} appears in {movie_title} {preposition} their co-star"
test_sentence_template = "Q: Who stars in a movie {preposition} {first_actor}? A: An actor named"

preposition = "with"

In [12]:
# FMFA ex #1
{"id": 1, "first_actor": "Melanie Lee", "second_actor": "Daniel Rose", "movie_title": "Inevitable Mixture", "main_character": "Jessica Ford", "release_year": 2029, "genre": "fantasy", "city": "Bowmanburgh", "box_office_earnings": 1}

# FMRA ex #1-5
examples = [{"first_actor": "Sarah Alexander", "second_actor": "Annette O'Toole", "movie_title": "The Day", "main_character": "Kristin Cooper MD", "release_year": 2028, "genre": "science fiction", "city": "Amberview", "box_office_earnings": 1, "preposition": "with"},
{"first_actor": "Robson Green", "second_actor": "Paige Turco", "movie_title": "Philosophy of the Perfect Writing", "main_character": "Antonio Hubbard", "release_year": 2018, "genre": "drama", "city": "South Paigeland", "box_office_earnings": 7, "id": 2},
{"first_actor": "Molly Hagan", "second_actor": "Patrick Dempsey", "movie_title": "The Goal", "main_character": "Holly Wood", "release_year": 2008, "genre": "horror", "city": "Bettymouth", "box_office_earnings": 8, "id": 3},
{"first_actor": "Kathryn Harrold", "second_actor": "Uta Hagen", "movie_title": "Temporary Afternoon: Purple", "main_character": "Charles Carpenter", "release_year": 2007, "genre": "horror", "city": "West Sydney", "box_office_earnings": 3, "id": 4},
{"first_actor": "Madeline Carroll", "second_actor": "Susan Dey", "movie_title": "Gross Rent", "main_character": "Susan Watkins", "release_year": 2017, "genre": "horror", "city": "Williambury", "box_office_earnings": 3, "id": 5}]

In [13]:
input_list = []

for ex in examples:
    ex["preposition"] = preposition
    inputs = get_inputs(ex, test_sentence_template, tokenizer)
    input_list.append(inputs)

for idx, token_idx in enumerate(inputs["input_ids"][0]):
    print(f"{idx}: {tokenizer.decode(token_idx)}")

# inputs, tokenizer.decode(inputs["input_ids"][0])

0: <bos>
1: Q
2: :
3:  Who
4:  stars
5:  in
6:  a
7:  movie
8:  with
9:  Madeline
10:  Carroll
11: ?
12:  A
13: :
14:  An
15:  actor
16:  named


In [14]:
generated_ids = llm_sft.generate(inputs["input_ids"], max_new_tokens=100)
generated_ids, tokenizer.decode(generated_ids[0])

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


(tensor([[     2, 235368, 235292,   7702,   8995,    575,    476,   7344,    675,
          152292,  47769, 235336,    586, 235292,   1364,  14250,   8602,  25729,
          109877, 235265,   4218,    575,   4968,   1944,   2528, 235269,    573,
           10198,   6397,  25729,  84176,    611,    476, 148542,  10734, 235265,
             109,  59959,  32485,   4887,  65066,    575, 235248, 235284, 235276,
          235274, 235324,    578,   3815,    611,    577,  14800,    697, 235304,
            4416,  42303, 235265,    714,   4592,  20245,    611,   1872,   3285,
           25729,  84176,    578,   1024,  10734, 235265,    109,  59959,  32485,
             729,  44848,   1337,    858,   8417,    575, 235248, 235284, 235276,
          235274, 235324,    578,    583,  21230,    697, 235304,   4416,  20455,
          235269,  39047,    476,   3779,   3741,   4844,   4665, 235265,    109,
           59959,  32485,    729,  44848,   1337,    858,   8417,    575, 235248,
          235284

In [15]:
# PATCH_CONFIG = "movie_attn_ffn_all_layers.yaml"
PATCH_CONFIG = "first_actor_attn_ffn_all_layers.yaml"
# PATCH_CONFIG = "preposition_attn_ffn_all_layers.yaml"
# PATCH_CONFIG = "no_patching.yaml"
# PATCH_CONFIG = "not_first_actor_preposition_attn_ffn_all_layers.yaml"
PATCH_CONFIG = "test_patching.yaml"

with open(PATCHES_DIR / PATCH_CONFIG, "r") as f:
    patch_config = yaml.safe_load(f)
patch_config = dict_to_namespace(patch_config)

In [41]:
patches_list = []

for ex, inputs in zip(examples, input_list):
    patches = get_patches(
        ex, patch_config, n_layers, tokenizer, inputs["input_ids"], test_sentence_template
    )
    patches_list.append(patches)
patches

2025-05-09 08:48:25,168 - INFO - Overriding previous patch at index 0 with token_idx
2025-05-09 08:48:25,170 - INFO - Overriding previous patch at index 0 with token_idx
2025-05-09 08:48:25,172 - INFO - Overriding previous patch at index 0 with token_idx
2025-05-09 08:48:25,177 - INFO - Overriding previous patch at index 0 with token_idx
2025-05-09 08:48:25,179 - INFO - Overriding previous patch at index 0 with token_idx


{0: Patch(patch_token_idx=0, patch_layers=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], targets=PatchTargets(embeddings=True, q=False, k=False, v=False, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 1: Patch(patch_token_idx=1, patch_layers=None, targets=PatchTargets(embeddings=False, q=True, k=True, v=True, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 2: Patch(patch_token_idx=2, patch_layers=None, targets=PatchTargets(embeddings=False, q=True, k=True, v=True, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 3: Patch(patch_token_idx=3, patch_layers=None, targets=PatchTargets(embeddings=False, q=True, k=True, v=True, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 4: Patch(patch_token_idx=4, patch_layers=None, targets=PatchTargets(embeddings=False, q=True, k=True, v=True, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True)),
 5: Patch(patch_token_idx=5, p

In [42]:
patch_direction = "sft2pre"
# patch_direction = "pre2sft"

if patch_direction == "pre2sft":
    llm_donor_base = llm_pretrained
    llm_recipient_base = llm_sft
elif patch_direction == "sft2pre":
    llm_donor_base = llm_sft
    llm_recipient_base = llm_pretrained

In [ ]:
probs_list = []

for inputs, patches in zip(input_list, patches_list):
    probs, dropout = run_patched_inference(
        inputs,
        patches,
        llm_donor_base,
        llm_recipient_base,
        model_config,
        tokenizer,
        # log_patches=True,
    )
    probs_list.append(probs)

2025-05-09 08:48:29,807 - INFO - Patching PatchTargets(embeddings=True, q=False, k=False, v=False, o=True, gate=True, mlp_up=True, mlp_down=True, ln_1=True, ln_2=True) at layer [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] for token idx 0: <bos>
2025-05-09 08:48:29,825 - INFO - Patching ln_1 at layer 0 for token idx 0
2025-05-09 08:48:29,826 - INFO - Patching ln_2 at layer 0 for token idx 0
2025-05-09 08:48:29,826 - INFO - Patching mlp_up at layer 0 for token idx 0
2025-05-09 08:48:29,827 - INFO - Patching mlp_down at layer 0 for token idx 0
2025-05-09 08:48:29,828 - INFO - Patching gate at layer 0 for token idx 0
2025-05-09 08:48:29,829 - INFO - Patching o at layer 0 for token idx 0
2025-05-09 08:48:29,830 - INFO - Patching ln_1 at layer 1 for token idx 0
2025-05-09 08:48:29,831 - INFO - Patching ln_2 at layer 1 for token idx 0
2025-05-09 08:48:29,832 - INFO - Patching mlp_up at layer 1 for token idx 0
2025-05-09 08:48:29,833 - INFO - Patching mlp_down at layer 1 for 

In [ ]:
target_key = "second_actor"
top_k = 5

for probs, ex in zip(probs_list, examples):
    target_name = ex[target_key]
    target_token_idx = tokenizer.encode(
        " " + target_name, add_special_tokens=False
    )[0]
    target_token = tokenizer.decode(target_token_idx)

    topk_probs, topk_indices = torch.topk(probs, top_k)
    target_token_prob = probs[target_token_idx].item()

    print(target_token, target_token_prob)

 Annette 0.8002858757972717
 Paige 0.9999823570251465
 Patrick 0.9966296553611755
 Uta 0.9999982118606567
 Susan 0.9999901056289673


In [38]:
for idx in range(top_k):
    print(f"{tokenizer.decode(topk_indices[idx])}: {topk_probs[idx].item()}")

 Susan: 0.9999901056289673
 Julie: 7.148660642997129e-06
Susan: 9.216214493790176e-07
 Su: 4.392373114114889e-07
 Mit: 3.6228806266080937e-07


In [372]:
# Output from pre2sft patching first actor
# ,: 0.2626367211341858
#  Michael: 0.03005307726562023
#  Jennifer: 0.019297853112220764
#  John: 0.014010615646839142
#  James: 0.01309494860470295